In [ ]:
pip install hyperopt

In [2]:
import numpy as np
import pandas as pd
import re
import datetime
import dateutil.relativedelta
from datetime import datetime
import math
from sklearn.model_selection import train_test_split

import h2o
from h2o.estimators import H2OXGBoostEstimator
from h2o.estimators import H2OGradientBoostingEstimator

from hyperopt import Trials, STATUS_OK, tpe, hp, fmin
from hyperopt.pyll import scope as ho_scope
from hyperopt.pyll.stochastic import sample as ho_sample

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', None)

import functions

In [3]:
from functions import rnkOrdering2,rnkOrdering2_con_wt,rnkOrdering2_ks_code,rnkOrderingN

In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,36 mins 37 secs
H2O_cluster_timezone:,UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,1 month and 11 days
H2O_cluster_name:,H2O_from_python_unknownUser_oyryg1
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.467 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [6]:
data = pd.read_csv('s3://scien-ds-sm-root-backup/home/vikashagarwal/LNV6/LNV5_Binned_Data_23Sept2024.csv')
data.shape

(3617, 140)

In [7]:
data.head(1)

,unique_id,addrlastmoveecontrajectory,addrchangecount60month,addrchangecount12month,addrprevioussubjectowned,addrcurrenttractratio,addrcurrentphoneservice,addrcurrentdeedmailing,addrcurrenttimeoldest,addrinputavmvalue12month,addrinputsubjectcount,addrinputtimenewest,addronfilecount,inquirynonshortterm12month,inquiryauto12month,bankruptcydismissed24month,bankruptcytimenewest,lienjudgmentdollartotal,lienjudgmentcount,evictioncount,assetpersonalcount,assetpersonal,assetpropeversoldcount,assetproppurchasetimeoldest,subjectnewestrecord12month,alertregulatorycondition,addrlastmovetaxratiodiff,addrchangecount03month,addrinputavmratio60monthprior,addrinputavmratio12monthprior,addronfilecollege,criminalfelonycount12month,assetproppurchasetimenewest,assetpropcurrentcount,auto_score,addrcurrentdwelltype,addrinputtimelastsale,addrinputphonecount,addronfilecorrectional,inquirytelcom12month,shorttermloanrequest24month,shorttermloanrequest,lienjudgmentothercount,lienjudgmentcourtcount,lienjudgmentcount12month,derogcount12month,derogcount,assetpropsaletimenewest,assetpropevercount,educationprogramattended,sourcecredheadertimeoldest,sourcenonderogcount12month,addrchangecount06month,addrpreviouslengthofres,addrcurrentcountyratio,addrcurrenttaxmarketvalue,addrinputblockratio,addrinputavmvalue,addrinputlengthofres,inquiryshortterm12month,lienjudgmenttaxcount,lienjudgmentsmallclaimscount,assetpropsalepurchaseratio,assetproppurchasecount12month,assetpropcurrenttaxtotal,businessassociation,profliccount,educationinstitutionprivate,sourcenonderogcount03month,addrpreviouscorrectional,addrcurrentcorrectional,addrcurrentavmratio60monthprior,addrinputcountyratio,addrinputdeedmailing,lienjudgmentforeclosurecount,evictioncount12month,criminalnonfelonycount,derogtimenewest,purchaseactivitydollartotal,assetpropnewestmortgagetype,assetownership,businesstitleleadership,educationinstitutionrating,sourcenonderogcount06month,subjectrecordtimeoldest,addrpreviousdwelltype,addrprevioustimenewest,addrcurrentavmratio12monthprior,addrcurrentsubjectowned,addrcurrentlengthofres,addrinputproblems,addrinputavmvalue60month,bankruptcycount24month,lienjudgmenttimenewest,criminalnonfelonycount12month,assetprop,educationattendance,sourcevoterregistration,crossindustry_score,addrprevioustimeoldest,addrcurrentblockratio,addrcurrentavmvalue12month,addrcurrenttaxvalue,addrcurrentlastsalesprice,addrinputtaxvalue,addrinputtimeoldest,bankruptcycount,evictiontimenewest,assetpropnewestsaleprice,assetpropsaletimeoldest,sourcenonderogcount,addrcurrentavmvalue60month,addrcurrenttimelastsale,addrcurrenttimenewest,addrinputtractratio,addrinputtaxmarketvalue,addrinputlastsaleprice,addrinputdwelltype,addronfilehighrisk,inquirycollections12month,shorttermloanrequest12month,bankruptcystatus,criminalnonfelonytimenewest,assetpropsoldcount12month,sourcecredheadertimenewest,subjectrecordtimenewest,bankcard_score,addrchangecount24month,addrcurrentavmvalue,addrinputphoneservice,addrinputsubjectowned,inquirybanking12month,bankruptcychapter,criminalfelonytimenewest,criminalfelonycount,businessassociationtimeoldest,proflictypecategory,educationevidence,bad_flag,funded_flag
0,2019-05-25 15:24|319-58-0454,"[-inf, 2.0)","[1.0, inf)","[1.0, inf)","[-inf, 1.0)","[1.08, inf)","[0.0612, inf)","[0.0, inf)","[-inf, 11.0)","[243465.0, inf)","[2.0, inf)","[-inf, 2.351)","[-inf, 2.0)","[-inf, 1.0)","[-inf, 0.0471)","[-0.0244, inf)","[-inf, -0.382)","[-inf, 911.144)","[-inf, 0.0391)","[-0.023, inf)","[-0.00443, inf)","[-0.0116, inf)","[-inf, 1.0)","[-inf, 3.0)","[0.937, inf)","[0.999, inf)","[-inf, 0.0)","[-inf, 0.0305)","[1.11, 1.49)","[1.02, inf)","[-0.00803, inf)","[-0.0238, inf)","[-inf, 3.0)","[-inf, 1.0)","[-inf, 658.0)",-1,"[-inf, 2.0)","[1.0, inf)","[-0.0247, inf)","[-inf, 0.00249)","[-0.0244, inf)","[-0.0208, inf)","[-0.0247, inf)","[-0.00166, inf)","[-0.0177, inf)","[-0.0116, inf)","[-inf, 0.0981)","[-inf, 10.0)","[-inf, 1.0)","[-inf, 0.0928)","[-inf, 61.0)","[-inf, 2.0)","[1.0, inf)","[-inf, 1.0)","[1.44, i

In [8]:
overall_target_mean = np.mean(data['bad_flag'])
overall_target_mean

0.021288360519767763

In [9]:
train, test= train_test_split(data, test_size= 0.3, random_state= 2, shuffle= True, stratify= data['bad_flag'])

In [10]:
train_target_mean = train.bad_flag.mean()
test_target_mean = test.bad_flag.mean()
train_target_mean,test_target_mean

(0.02133544053733702, 0.02117863720073665)

In [11]:
Train_h2o = h2o.H2OFrame(train)
Test_h2o = h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [12]:
Train_h2o.shape,Test_h2o.shape

((2531, 140), (1086, 140))

In [16]:
if 'wt2' not in Train_h2o.columns:
    Train_h2o['wt2'] = 1
    Test_h2o['wt2'] = 1

In [17]:
Train_h2o[0,:]

unique_id,addrlastmoveecontrajectory,addrchangecount60month,addrchangecount12month,addrprevioussubjectowned,addrcurrenttractratio,addrcurrentphoneservice,addrcurrentdeedmailing,addrcurrenttimeoldest,addrinputavmvalue12month,addrinputsubjectcount,addrinputtimenewest,addronfilecount,inquirynonshortterm12month,inquiryauto12month,bankruptcydismissed24month,bankruptcytimenewest,lienjudgmentdollartotal,lienjudgmentcount,evictioncount,assetpersonalcount,assetpersonal,assetpropeversoldcount,assetproppurchasetimeoldest,subjectnewestrecord12month,alertregulatorycondition,addrlastmovetaxratiodiff,addrchangecount03month,addrinputavmratio60monthprior,addrinputavmratio12monthprior,addronfilecollege,criminalfelonycount12month,assetproppurchasetimenewest,assetpropcurrentcount,auto_score,addrcurrentdwelltype,addrinputtimelastsale,addrinputphonecount,addronfilecorrectional,inquirytelcom12month,shorttermloanrequest24month,shorttermloanrequest,lienjudgmentothercount,lienjudgmentcourtcount,lienjudgmentcount12month,derogcount12month,derogcount,assetpropsaletimenewest,assetpropevercount,educationprogramattended,sourcecredheadertimeoldest,sourcenonderogcount12month,addrchangecount06month,addrpreviouslengthofres,addrcurrentcountyratio,addrcurrenttaxmarketvalue,addrinputblockratio,addrinputavmvalue,addrinputlengthofres,inquiryshortterm12month,lienjudgmenttaxcount,lienjudgmentsmallclaimscount,assetpropsalepurchaseratio,assetproppurchasecount12month,assetpropcurrenttaxtotal,businessassociation,profliccount,educationinstitutionprivate,sourcenonderogcount03month,addrpreviouscorrectional,addrcurrentcorrectional,addrcurrentavmratio60monthprior,addrinputcountyratio,addrinputdeedmailing,lienjudgmentforeclosurecount,evictioncount12month,criminalnonfelonycount,derogtimenewest,purchaseactivitydollartotal,assetpropnewestmortgagetype,assetownership,businesstitleleadership,educationinstitutionrating,sourcenonderogcount06month,subjectrecordtimeoldest,addrpreviousdwelltype,addrprevioustimenewest,addrcurrentavmratio12monthprior,addrcurrentsubjectowned,addrcurrentlengthofres,addrinputproblems,addrinputavmvalue60month,bankruptcycount24month,lienjudgmenttimenewest,criminalnonfelonycount12month,assetprop,educationattendance,sourcevoterregistration,crossindustry_score,addrprevioustimeoldest,addrcurrentblockratio,addrcurrentavmvalue12month,addrcurrenttaxvalue,addrcurrentlastsalesprice,addrinputtaxvalue,addrinputtimeoldest,bankruptcycount,evictiontimenewest,assetpropnewestsaleprice,assetpropsaletimeoldest,sourcenonderogcount,addrcurrentavmvalue60month,addrcurrenttimelastsale,addrcurrenttimenewest,addrinputtractratio,addrinputtaxmarketvalue,addrinputlastsaleprice,addrinputdwelltype,addronfilehighrisk,inquirycollections12month,shorttermloanrequest12month,bankruptcystatus,criminalnonfelonytimenewest,assetpropsoldcount12month,sourcecredheadertimenewest,subjectrecordtimenewest,bankcard_score,addrchangecount24month,addrcurrentavmvalue,addrinputphoneservice,addrinputsubjectowned,inquirybanking12month,bankruptcychapter,criminalfelonytimenewest,criminalfelonycount,businessassociationtimeoldest,proflictypecategory,educationevidence,bad_flag,funded_flag,wt2
2019-09-11 15:13|535-88-5202,"[2.0, 11.0)","[1.0, inf)","[1.0, inf)","[1.0, inf)","[-inf, 0.26)","[-inf, 0.0612)","[0.0, inf)","[-inf, 11.0)","[243465.0, inf)","[2.0, inf)","[-inf, 2.351)","[3.0, 5.0)","[1.0, inf)","[0.0471, inf)","[-0.0244, inf)","[-0.382, inf)","[911.144, inf)","[0.0391, inf)","[-0.023, inf)","[-0.00443, inf)","[-0.0116, inf)","[1.0, inf)","[3.0, 162.0)","[0.937, inf)","[0.999, inf)","[0.0, inf)","[0.0305, inf)","[1.78, inf)","[1.02, inf)","[-0.00803, inf)","[-0.0238, inf)","[79.0, inf)","[-inf, 1.0)","[658.0, 684.0)",H,"[-inf, 2.0)","[-inf, 1.0)","[-0.0247, inf)","[-inf, 0.00249)","[-0.0244, inf)","[-0.0208, inf)","[-0.0247, inf)","[-0.00166, inf)","[-0.0177, inf)","[-0.0116, inf)","[0.0981, inf)","[10.0, inf)","[1.0, 2.0)","[-inf, 0.0928)","[202.0, 398.0)","[-inf, 2.0)","[1.0, inf)","[17.0, 52.0)","[-inf, 0.49)",

In [18]:
bad_col = "bad_flag"
unique_cols = ['unique_id', 'funded_flag']
wt_col = "wt2"

In [29]:
Train_h2o[bad_col] = Train_h2o[bad_col].asfactor() # Convert column in the current frame to categoricals.
folds = 10
predictors = Train_h2o.columns
predictors.remove('unique_id')
predictors.remove('funded_flag')
len(predictors)

139

In [30]:
def Bucket_Order(data):
    data['lower_bound'] = data['Bucket'].str.replace('[','').str.replace(']','').str.replace('(','').str.replace(')','').str.split(',').str[0]
    data['lower_bound'] = data['lower_bound'].apply(lambda x: -np.inf if x == '-inf' else np.inf if x == 'inf' else float(x))
    data = data.sort_values('lower_bound')
    a = data.loc[data['lower_bound']=="[-99999, -99998, -99997]"]
    b = data.loc[data['lower_bound']!="[-99999, -99998, -99997]"]
    b['Is_Trend']= 'Yes' if (
            (b['Train_BadRate'].is_monotonic_decreasing and b['Test_BadRate'].is_monotonic_decreasing) or
            (b['Train_BadRate'].is_monotonic_increasing and b['Test_BadRate'].is_monotonic_increasing)
        ) else 'No'
    data = pd.concat(objs=[a,b])
    data = data.drop(columns=['lower_bound'])
    return data

In [31]:
# df_bv= pd.DataFrame()

# for col in predictors:
#     if col in ['unique_id','wt2','bad_flag'] :
#         continue

#     ### Train
#     temp_tr= train.groupby(col).agg(bad_rate_tr= ('bad_flag', 'mean'),
#                                       bad_tr= ('bad_flag', 'sum'),
#                                      volume_tr= ('unique_id', 'count')).reset_index()
#     temp_tr.columns= ['Bucket', 'Train_BadRate','Train_Bad','Train_#_Apps']
#     temp_tr['Variable']= col
#     temp_tr['Train_%_Apps'] = temp_tr['Train_#_Apps'] / temp_tr['Train_#_Apps'].sum()

#     ### Test
#     temp_te= test.groupby(col).agg(bad_rate_te= ('bad_flag', 'mean'),
#                                      bad_te= ('bad_flag', 'sum'),
#                                     volume_tr= ('unique_id', 'count')).reset_index()
#     temp_te.columns= ['Bucket', 'Test_BadRate','Test_Bad','Test_#_Apps']
#     temp_te['Test_%_Apps'] = temp_te['Test_#_Apps'] / temp_te['Test_#_Apps'].sum()

#     ### Concatenating
#     temp= temp_tr.merge(temp_te, on= ['Bucket'], how= 'left')
#     temp = Bucket_Order(temp)
    
# #     display(temp)
#     df_bv= pd.concat([df_bv, temp]).reset_index(drop= True)
#     df_bv = df_bv[['Variable','Bucket','Train_BadRate','Train_Bad','Train_#_Apps','Train_%_Apps','Test_BadRate','Test_Bad','Test_#_Apps','Test_%_Apps','Is_Trend']]
#     del temp, temp_tr, temp_te

In [32]:
# print(df_bv.shape)
# display(df_bv.head())

In [33]:
# Selected_Columns = list(set(df_bv[df_bv['Is_Trend']=='Yes']['Variable'].to_list()))
# print(len(Selected_Columns))
# # Selected_Columns

In [34]:
# Rejected_Columns = list(set(df_bv[df_bv['Is_Trend']=='No']['Variable'].to_list()))
# print(len(Rejected_Columns))
# # Rejected_Columns

In [35]:
model = H2OXGBoostEstimator(seed = 42)

In [36]:
model.train(y = bad_col, x = predictors, training_frame = Train_h2o, weights_column = wt_col)

xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_model_python_1728457214358_12755


Model Summary: 
    number_of_trees
--  -----------------
    50

ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.002503842498216215
RMSE: 0.050038410228705456
LogLoss: 0.015406498196110188
Mean Per-Class Error: 0.009259259259259259
AUC: 0.9973010960092107
AUCPR: 0.9838760186330897
Gini: 0.9946021920184214

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.2592085301876068
       0     1    Error    Rate
-----  ----  ---  -------  ------------
0      2477  0    0        (0.0/2477.0)
1      1     53   0.0185   (1.0/54.0)
Total  2478  53   0.0004   (1.0/2531.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.259209     0.990654  50
max f2                       0.259209     0.98513   50
max f0point5                 0.259209     0.996241  50
max accuracy                 0.259209     0.999605  50
max precision                0.874748     1         0
max recall                   0.011229     1         238
max specificity              0.874748     1         0
max absolute_mcc             0.259209     0.990498  50
max min_per_class_accuracy   0.259209     0.981481  50
max mean_per_class_accuracy  0.259209     0.990741  50
max tns                      0.874748     2477      0
max fns                      0.874748     53        0
max fps                      3.83031e-05  2477      399
max tps                      0.011229     54        238
max tnr                      0.874748     1         0
max fnr                      0.874748     0.981481  0
max fpr                      3.83031e-05  1         399
max tpr                      0.011229     1         238

Gains/Lift Table: Avg response rate:  2.13 %, avg score:  2.16 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0102726                   0.75435            46.8704   46.8704            1                0.802755     1                           0.802755            0.481481        0.481481                   4587.04   4587.04            0.481481
2        0.0201501                   0.517015           46.8704   46.8704            1                0.668938     1                           0.737159            0.462963        0.944444                   4587.04   4587.04            0.944444
3        0.0300277                   0.0790843          3.74963   32.6859            0.08             0.142962     0.697368                    0.541699            0.037037        0.981481                   274.963   3168.59            0.972196
4        0.0403003                   0.0568666          0         24.3542            0                0.0655922    0.519608                    0.420339            0               0.981481                   -100      2335.42            0.961699
5        0.0501778                   0.043008           0         19.5601            0                0.0489851    0.417323                    0.347238            0               0.981481                   -100      1856.01            0.951607
6        0.100356                    0.0201497          0         9.78004            0                0.0283325    0.208661                    0.187785            0               0.981481                   -100      878.004            0.900335
7        0.150138                    0.0113325          0         6.53718            0   

### TRAIN

In [37]:
predict = model.predict(Train_h2o)
p =predict.as_data_frame()
dd = Train_h2o[:,unique_cols + [wt_col] + [bad_col]].as_data_frame()
dd['RECWGT'] = dd[wt_col]
dd['conwt'] = 1
pred = pd.concat([dd.loc[:,(bad_col,"RECWGT","conwt")],p],axis=1)
ks_dict, ks_df = rnkOrderingN(pred, bad_col, "RECWGT","p1", 5)
# rnkOrdering2_ks_train_nw = rnkOrdering2_ks_code(pred, bad_col, "conwt","p1", 5)
print(ks_dict)
ks_df

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
{'Gini': 81.71000000000001, 'KS': 81.71}


,min_score,max_score,mean_score,total,Goods,Bads,Bad_Rate,Cum_Total,Cum_Good,Cum_Bad,KS,Bad_Rate_Diff
1,0.009547,0.874748,0.100539,507,453,54,10.65,20.03,18.29,100.0,81.71,0.596054
2,0.002403,0.009537,0.005083,506,506,0,0.00,40.02,38.72,100.0,61.28,0.508325
3,0.000779,0.002398,0.001434,506,506,0,0.00,60.02,59.14,100.0,40.86,0.143443
4,0.000269,0.000778,0.000493,506,506,0,0.00,80.01,79.57,100.0,20.43,0.049298
5,0.000022,0.000269,0.000150,506,506,0,0.00,100.00,100.00,100.0,0.00,0.014980


### TEST

In [38]:
predict = model.predict(Test_h2o)
p =predict.as_data_frame()
dd = Test_h2o[:,unique_cols + [wt_col] + [bad_col]].as_data_frame()
dd['RECWGT'] = dd[wt_col]
dd['conwt'] = 1
pred = pd.concat([dd.loc[:,(bad_col,"RECWGT","conwt")],p],axis=1)
ks_dict, ks_df = rnkOrderingN(pred, bad_col, "RECWGT","p1", 5)
print(ks_dict)
ks_df

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
{'Gini': 44.330000000000005, 'KS': 32.79}


,min_score,max_score,mean_score,total,Goods,Bads,Bad_Rate,Cum_Total,Cum_Good,Cum_Bad,KS,Bad_Rate_Diff
1,0.011229,0.825141,0.062278,218,206,12,5.50,20.07,19.38,52.17,32.79,0.727826
2,0.002716,0.011229,0.005592,217,213,4,1.84,40.06,39.42,69.57,30.15,1.280827
3,0.000744,0.002701,0.001492,217,212,5,2.30,60.04,59.36,91.30,31.94,2.150804
4,0.000239,0.000742,0.000427,217,216,1,0.46,80.02,79.68,95.65,15.97,0.417270
5,0.000027,0.000239,0.000134,217,216,1,0.46,100.00,100.00,100.00,0.00,0.446635


In [39]:
var_imp = model._model_json['output']['variable_importances'].as_data_frame()
var_imp['Var_unique'] = var_imp['variable'].str.split('.').str[0]
# var_imp

In [40]:
var_imp_list_baseline1 = list(var_imp['Var_unique'].unique())
print(len(var_imp_list_baseline1))
# print(var_imp_list_baseline1)

77


In [41]:
Var_Imp_DF = var_imp.groupby('Var_unique').agg({'percentage':'sum'}).reset_index().sort_values(by='percentage',ascending=False).reset_index(drop=True)
Var_Imp_DF.head()

,Var_unique,percentage
0,crossindustry_score,0.045069
1,addronfilecount,0.042500
2,addrprevioustimenewest,0.041045
3,addrinputdeedmailing,0.040523
4,addrinputavmvalue,0.039694


## HyperOpt

In [42]:
from IPython.display import clear_output

def get_predictions(data_h2o, bad_col, wt_col, classifier):
    predict = classifier.predict(data_h2o)
    p = predict.as_data_frame()
    dd = data_h2o[:, unique_cols + [bad_col] + [wt_col]].as_data_frame()
    dd['RECWGT'] = dd[wt_col]
    dd['con_wt'] = 1
    pred = pd.concat([dd, p], axis=1)
    return pred

def objective(space):
    ks_data = pd.DataFrame()
    from sklearn.metrics import roc_auc_score

    classifier = H2OXGBoostEstimator(ntrees = int(space['n_estimators']),
                                max_depth = int(space['max_depth']),
                                learn_rate = space['learning_rate'],
                                min_rows = space['min_child_weight'],
                                # sample_rate = space['subsample'],
                                # col_sample_rate_per_tree = space['colsample_bytree'],
                                reg_alpha = space['reg_alpha'],
                                reg_lambda = space['reg_lambda'],
                                min_split_improvement = space['gamma'],
                                # col_sample_rate = space['colsample_bylevel'],
                                seed=space['seed'],
                                model_id="Scienaptic_LN_V6_Model_1",
#                                 nfolds = 10,
#                                 distribution = "bernoulli",
#                                 stopping_tolerance=0.000001, 
#                                 stopping_rounds=20,
#                                 stopping_metric = 'logloss',
#                                 categorical_encoding ='sort_by_response'
                                )
    
    h2o.no_progress()
    classifier.train(y=bad_col, x = predictors, training_frame=Train_h2o, weights_column =wt_col)
    # h2o.show_progress()
    
    ################        TRAIN      ##########################
    
    pred = get_predictions(Train_h2o, bad_col, wt_col, classifier)

    y_true = pred[bad_col].values
    y_prob = pred['p1'].values
    weights = pred['RECWGT'].values
    auc_train = roc_auc_score(y_true, y_prob, sample_weight=weights)
    
    NonWeighted_Dict, NonWeighted_KS_Table = rnkOrderingN(pred, bad_col, "con_wt","p1", 5) # Non Weighted
    # Weighted_Dict, Weighted_KS_Table = rnkOrderingN(pred, bad_col, "RECWGT","p1", 5) # Weighted
    
    Is_NW_Train_Bad_Rate_RankOrder = NonWeighted_KS_Table['Bad_Rate'].is_monotonic_decreasing
    NonWeighted_KS_Table['on'] = 'Train'
    NonWeighted_KS_Table['iter'] = len(trials.trials)
    NonWeighted_KS_Table['bins'] = len(NonWeighted_KS_Table)
#     NonWeighted_KS_Table['Weighted'] = 'No'
    
#     Is_Train_Bad_Rate_RankOrder = Weighted_KS_Table['Bad_Rate'].is_monotonic_decreasing
#     Weighted_KS_Table['on'] = 'Train'
#     Weighted_KS_Table['iter'] = len(trials.trials)
#     Weighted_KS_Table['bins'] = len(Weighted_KS_Table)
#     Weighted_KS_Table['Weighted'] = 'Yes'
    # ks_data = pd.concat([ks_data,NonWeighted_KS_Table,Weighted_KS_Table])
    
    ks_data = pd.concat([ks_data,NonWeighted_KS_Table])
    
#     train_ks = Weighted_Dict['KS']
#     train_gini = Weighted_Dict['Gini']

    NW_train_ks = NonWeighted_Dict['KS']
    NW_train_gini = NonWeighted_Dict['Gini']
    
     ################        TEST       ##########################
    
    pred = get_predictions(Test_h2o, bad_col, wt_col, classifier)
    
    y_true = pred[bad_col].values
    y_prob = pred['p1'].values
    weights = pred['RECWGT'].values
    auc_test = roc_auc_score(y_true, y_prob, sample_weight=weights)
    
    NonWeighted_Dict, NonWeighted_KS_Table = rnkOrderingN(pred, bad_col, "con_wt","p1", 5) # Non Weighted
    # Weighted_Dict, Weighted_KS_Table = rnkOrderingN(pred, bad_col, "RECWGT","p1", 5) # Weighted
    
    Is_NW_Test_Bad_Rate_RankOrder = NonWeighted_KS_Table['Bad_Rate'].is_monotonic_decreasing
    NonWeighted_KS_Table['on'] = 'Test'
    NonWeighted_KS_Table['iter'] = len(trials.trials)
    NonWeighted_KS_Table['bins'] = len(NonWeighted_KS_Table)
#     NonWeighted_KS_Table['Weighted'] = 'No'
    
#     Is_Test_Bad_Rate_RankOrder = Weighted_KS_Table['Bad_Rate'].is_monotonic_decreasing
#     Weighted_KS_Table['on'] = 'Test'
#     Weighted_KS_Table['iter'] = len(trials.trials)
#     Weighted_KS_Table['bins'] = len(Weighted_KS_Table)
#     Weighted_KS_Table['Weighted'] = 'Yes'
    # ks_data = pd.concat([ks_data,NonWeighted_KS_Table,Weighted_KS_Table])
    
    ks_data = pd.concat([ks_data,NonWeighted_KS_Table])
    
    # test_ks = Weighted_Dict['KS']
    # test_gini = Weighted_Dict['Gini']

    NW_test_ks = NonWeighted_Dict['KS']
    NW_test_gini = NonWeighted_Dict['Gini']

    # KS_diff = abs(train_ks-test_ks)
    # gini_diff = abs(train_gini-test_gini)
    NW_ks_diff = abs(NW_train_ks-NW_test_ks)
    NW_gini_diff = abs(NW_train_gini-NW_test_gini)
    
    print(f'{len(trials.trials)}, ks_diff_nw: {NW_ks_diff}')

    auc_diff = abs(auc_train-auc_test) 
    
    loss = NW_ks_diff
    
    return{'loss':NW_ks_diff, 'status': STATUS_OK ,
               # 'ks_diff':KS_diff,'train_ks':train_ks,'test_ks':test_ks, #'oot_ks':oot_ks,    ###  WEIGHTED KS
               # 'gini_diff':gini_diff,'train_gini':train_gini,'test_gini':test_gini, #'oot_gini':oot_gini,     ###  WEIGHTED GINI
               'NW_ks_diff':NW_ks_diff,'NW_train_ks':NW_train_ks,'NW_test_ks':NW_test_ks,# 'oot_ks_cw':oot_ks_cw,   ### NON_WEIGHTED KS
               'NW_gini_diff':NW_gini_diff,'NW_train_gini':NW_train_gini,'NW_test_gini':NW_test_gini #'oot_gini_cw':oot_gini_cw   ###   NON-WEIGHTED GINI
              }

In [46]:
space = {
    'max_depth':2,
    'learning_rate' : hp.quniform('learning_rate',0.01,0.05,0.001),
    'n_estimators' : ho_scope.int(hp.quniform('n_estimators',100,1000,25)),
    'min_child_weight' : hp.quniform('min_child_weight', 2, 32, 2),
    # 'subsample' : hp.quniform('subsample', 0.1, 0.6, 0.05),
    # 'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'reg_lambda':  hp.quniform('reg_lambda',1,15,0.5),
    'reg_alpha': hp.quniform('reg_alpha',1,10,0.5),
    'gamma':hp.quniform('gamma',1,25,0.5),
    # 'colsample_bylevel':hp.quniform('colsample_bylevel',0.5, 1, 0.05),
    'seed':42
}

In [47]:
trials = Trials()

In [49]:
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials)

print("Best: ", best)

1, ks_diff_nw: 3.009999999999998                      
2, ks_diff_nw: 6.200000000000003                                               
3, ks_diff_nw: 5.219999999999999                                               
4, ks_diff_nw: 0.7700000000000031                                              
5, ks_diff_nw: 3.989999999999995                                                
6, ks_diff_nw: 3.989999999999995                                                
7, ks_diff_nw: 3.2300000000000004                                               
8, ks_diff_nw: 3.2300000000000004                                               
9, ks_diff_nw: 3.009999999999998                                                
10, ks_diff_nw: 3.2300000000000004                                              
11, ks_diff_nw: 2.5700000000000003                                               
12, ks_diff_nw: 6.829999999999998                                                
13, ks_diff_nw: 3.2300000000000004                     

In [50]:
best

{'gamma': 2.0,
 'learning_rate': 0.022,
 'min_child_weight': 4.0,
 'n_estimators': 200.0,
 'reg_alpha': 6.0,
 'reg_lambda': 3.5}

In [51]:
tr_results = pd.DataFrame(trials.results).reset_index()
tr_results['iter'] = tr_results['index']+1
print(f'TR_RESULTS = {tr_results.shape}')
display(tr_results.head())

tr_vals = pd.DataFrame(trials.vals).reset_index()
tr_vals['iter'] = tr_vals['index']+1
print(f'TR_VALUES = {tr_vals.shape}')
display(tr_vals.head())

trials_full = pd.merge(tr_vals.drop('index',axis=1),tr_results.drop('index',axis=1),how='left',on='iter')
print(f'Trials Full = {trials_full.shape}')
display(trials_full.head())

TR_RESULTS = (50, 10)


,index,loss,status,NW_ks_diff,NW_train_ks,NW_test_ks,NW_gini_diff,NW_train_gini,NW_test_gini,iter
0,0,3.01,ok,3.01,40.47,43.48,4.81,48.40,53.21,1
1,1,6.20,ok,6.20,46.15,52.35,1.97,60.51,58.54,2
2,2,5.22,ok,5.22,44.25,39.03,6.05,57.48,51.43,3
3,3,0.77,ok,0.77,44.25,43.48,0.99,55.97,54.98,4
4,4,3.99,ok,3.99,38.58,34.59,1.33,38.56,37.23,5


TR_VALUES = (50, 8)


,index,gamma,learning_rate,min_child_weight,n_estimators,reg_alpha,reg_lambda,iter
0,0,8.5,0.047,16.0,325.0,8.0,12.0,1
1,1,3.5,0.050,10.0,850.0,2.5,4.0,2
2,2,2.0,0.035,24.0,250.0,6.5,11.5,3
3,3,8.5,0.036,22.0,350.0,2.0,5.5,4
4,4,18.5,0.049,10.0,975.0,3.5,7.0,5


Trials Full = (50, 15)


,gamma,learning_rate,min_child_weight,n_estimators,reg_alpha,reg_lambda,iter,loss,status,NW_ks_diff,NW_train_ks,NW_test_ks,NW_gini_diff,NW_train_gini,NW_test_gini
0,8.5,0.047,16.0,325.0,8.0,12.0,1,3.01,ok,3.01,40.47,43.48,4.81,48.40,53.21
1,3.5,0.050,10.0,850.0,2.5,4.0,2,6.20,ok,6.20,46.15,52.35,1.97,60.51,58.54
2,2.0,0.035,24.0,250.0,6.5,11.5,3,5.22,ok,5.22,44.25,39.03,6.05,57.48,51.43
3,8.5,0.036,22.0,350.0,2.0,5.5,4,0.77,ok,0.77,44.25,43.48,0.99,55.97,54.98
4,18.5,0.049,10.0,975.0,3.5,7.0,5,3.99,ok,3.99,38.58,34.59,1.33,38.56,37.23


In [54]:
# trials_full

In [79]:
iter = 28
dictionary = trials_full[trials_full['iter']==iter].loc[iter-1,'gamma':'reg_lambda'].to_dict()
dictionary.update({'seed':42})
ntrees = dictionary['n_estimators']
learn_rate = dictionary['learning_rate']
dictionary['ntrees'] = int(ntrees)
dictionary['learn_rate'] = learn_rate
del dictionary['n_estimators']
del dictionary['learning_rate']
dictionary['max_depth'] = 2
dictionary

{'gamma': 1.5,
 'min_child_weight': 20.0,
 'reg_alpha': 3.0,
 'reg_lambda': 4.0,
 'seed': 42,
 'ntrees': 600,
 'learn_rate': 0.025,
 'max_depth': 2}

In [80]:
best_model = H2OXGBoostEstimator(**dictionary)

In [81]:
best_model.train(y = bad_col, x = predictors, training_frame = Train_h2o, weights_column = wt_col)

Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_model_python_1728457214358_19575


Model Summary: 
    number_of_trees
--  -----------------
    600

ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.020278170406513252
RMSE: 0.14240144102681423
LogLoss: 0.09198764291904367
Mean Per-Class Error: 0.395953886870318
AUC: 0.808964697438658
AUCPR: 0.11811721480105429
Gini: 0.6179293948773159

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.0629379078745842
       0     1    Error    Rate
-----  ----  ---  -------  -------------
0      2442  35   0.0141   (35.0/2477.0)
1      42    12   0.7778   (42.0/54.0)
Total  2484  47   0.0304   (77.0/2531.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.0629379    0.237624  1
max f2                       0.0549776    0.261097  9
max f0point5                 0.0678454    0.252525  0
max accuracy                 0.0678454    0.972343  0
max precision                0.0678454    0.277778  0
max recall                   0.00995057   1         248
max specificity              0.0678454    0.989503  0
max absolute_mcc             0.0629379    0.222736  1
max min_per_class_accuracy   0.0275268    0.726686  101
max mean_per_class_accuracy  0.0275268    0.733713  101
max tns                      0.0678454    2451      0
max fns                      0.0678454    44        0
max fps                      0.00879159   2477      258
max tps                      0.00995057   54        248
max tnr                      0.0678454    0.989503  0
max fnr                      0.0678454    0.814815  0
max fpr                      0.00879159   1         258
max tpr                      0.00995057   1         248

Gains/Lift Table: Avg response rate:  2.13 %, avg score:  2.25 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0142236                   0.0678454          13.0195   13.0195            0.277778         0.0678454   0.277778                    0.0678454           0.185185        0.185185                   1201.95   1201.95            0.174689
2        0.0323983                   0.0618454          3.05676   7.43067            0.0652174        0.0621236   0.158537                    0.0646356           0.0555556       0.240741                   205.676   643.067            0.212884
3        0.0438562                   0.0585227          3.23244   6.33383            0.0689655        0.058649    0.135135                    0.0630715           0.037037        0.277778                   223.244   533.383            0.239021
4        0.0561043                   0.0575874          3.02389   5.61124            0.0645161        0.0575874   0.119718                    0.0618743           0.037037        0.314815                   202.389   461.124            0.264351
5        0.122086                    0.0495993          1.96463   3.64042            0.0419162        0.0518168   0.0776699                   0.0564387           0.12963         0.444444                   96.4626   264.042            0.329386
6        0.155275                    0.0433255          2.7899    3.45863            0.0595238        0.0452224   0.0737913                   0.0540413           0.0925926       0.537037                   178.99    245.863            0.390085
7        0.200711                    0.0364511          1.63027   3.04473            0.0347826      

### TRAIN

In [82]:
predict = best_model.predict(Train_h2o)
p =predict.as_data_frame()
dd = Train_h2o[:,unique_cols + [wt_col] + [bad_col]].as_data_frame()
dd['RECWGT'] = dd[wt_col]
dd['conwt'] = 1
pred = pd.concat([dd.loc[:,(bad_col,"RECWGT","conwt")],p],axis=1)
ks_dict, ks_df = rnkOrderingN(pred, bad_col, "RECWGT","p1", 5)
# rnkOrdering2_ks_train_nw = rnkOrdering2_ks_code(pred, bad_col, "conwt","p1", 5)
print(ks_dict)
ks_df

{'Gini': 58.24, 'KS': 44.25}


,min_score,max_score,mean_score,total,Goods,Bads,Bad_Rate,Cum_Total,Cum_Good,Cum_Bad,KS,Bad_Rate_Diff
1,0.036451,0.067845,0.050531,507,474,33,6.51,20.03,19.14,61.11,41.97,1.456936
2,0.021135,0.036451,0.027160,506,494,12,2.37,40.02,39.08,83.33,44.25,0.345954
3,0.013110,0.021135,0.015439,506,498,8,1.58,60.02,59.18,98.15,38.97,0.036130
4,0.009425,0.013110,0.010505,506,505,1,0.20,80.01,79.57,100.00,20.43,0.850542
5,0.008792,0.009350,0.008916,506,506,0,0.00,100.00,100.00,100.00,0.00,0.891589


### TEST

In [83]:
predict = best_model.predict(Test_h2o)
p =predict.as_data_frame()
dd = Test_h2o[:,unique_cols + [wt_col] + [bad_col]].as_data_frame()
dd['RECWGT'] = dd[wt_col]
dd['conwt'] = 1
pred = pd.concat([dd.loc[:,(bad_col,"RECWGT","conwt")],p],axis=1)
ks_dict, ks_df = rnkOrderingN(pred, bad_col, "RECWGT","p1", 5)
print(ks_dict)
ks_df

{'Gini': 54.99000000000001, 'KS': 43.48}


,min_score,max_score,mean_score,total,Goods,Bads,Bad_Rate,Cum_Total,Cum_Good,Cum_Bad,KS,Bad_Rate_Diff
1,0.036629,0.067845,0.049679,218,204,14,6.42,20.07,19.19,60.87,41.68,1.452061
2,0.021135,0.036629,0.026760,217,212,5,2.30,40.06,39.13,82.61,43.48,0.376039
3,0.013392,0.021135,0.015697,217,214,3,1.38,60.04,59.27,95.65,36.38,0.189748
4,0.009333,0.013392,0.010477,217,217,0,0.00,80.02,79.68,95.65,15.97,1.047718
5,0.008792,0.009333,0.008851,217,216,1,0.46,100.00,100.00,100.00,0.00,0.425058


In [84]:
var_imp = best_model._model_json['output']['variable_importances'].as_data_frame()
var_imp['Var_unique'] = var_imp['variable'].str.split('.').str[0]
# var_imp

In [85]:
var_imp_list_baseline1 = list(var_imp['Var_unique'].unique())
print(len(var_imp_list_baseline1))
# print(var_imp_list_baseline1)

11


In [86]:
Var_Imp_DF = var_imp.groupby('Var_unique').agg({'percentage':'sum'}).reset_index().sort_values(by='percentage',ascending=False).reset_index(drop=True)
Var_Imp_DF

,Var_unique,percentage
0,addrinputavmvalue,0.347467
1,bankcard_score,0.221058
2,assetpropevercount,0.099186
3,addrinputblockratio,0.070234
4,addrcurrentdwelltype,0.051293
5,sourcenonderogcount12month,0.046355
6,addrinputdeedmailing,0.044485
7,addrprevioustimenewest,0.036471
8,addrlastmovetaxratiodiff,0.033706
9,addrcurrenttaxvalue,0.025269
